# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [1]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [2]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/UR_NSYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [3]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)
persistDirectory = './data/processed/faq_data/vectordata/UR_NSYR'
vectorstore = Chroma.from_documents(
    documents=data, embedding=embeddings, persist_directory=persistDirectory)
vectorstore.persist()

Run the cells below to re-use already-generated vector embeddings.

In [4]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/UR_NSYR'
# Retrieve and generate answers using relevant FAQs
vectordb = Chroma(persist_directory=persistDirectory,
                  embedding_function=embeddings)

retriever = vectordb.as_retriever()

In [5]:
from utils import CreateInferenceModifier # Import the function from utils.py
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """Drawing from your knowledge base, answer the question below.
If you don't know the answer from the provided context, tell me that your training materials don't include this information.
Keep the answer as concise and relevant to the question as possible.

Knowledge Base: {context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template)


# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4000,
             "temperature": 0.2,
             "top_k": 250,
             "top_p": 1,
             "stop_sequences": ["\n\nHuman"],
             }


llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=CreateInferenceModifier("claude", params=modifiers))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [6]:
# Define a function to extract unique URLs used in the retrieved source materials.
from IPython.display import Markdown, display
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string


# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke(
    "ترکی میں پناہ کی درخواست دینے کے لیے مجھے کیا اقدامات کرنے کی ضرورت ہے؟")
answer = response["answer"]
prettyResponse = f"## Answer: \n\n {answer} \n\n ## Sources: \n\n {extract_unique_urls(response)}"
Markdown(prettyResponse)

## Answer: 

  ترکی میں پناہ کی درخواست دینے کے لیے آپ کو مندرجہ ذیل اقدامات کرنے کی ضرورت ہے:

1. آپ کو اپنی پناہ کی درخواست جلد از جلد ترکی پہنچنے کے بعد جمع کرانی چاہیے۔ کوئی قانونی وقت کی حد تو مقرر نہیں ہے لیکن جلدی سے درخواست دینا بہتر ہے۔

2. آپ کو اپنی درخواست صوبائی ڈائریکٹوریٹ آف مائیگریشن مینجمنٹ کے سامنے پیش کرنی ہوگی جہاں آپ رجسٹرڈ ہیں۔

3. آپ کو اپنی درخواست میں یہ بتانا ہوگا کہ آپ کیوں اپنے ملک سے بھاگے ہیں اور آپ کو وہاں کیا خطرات لاحق ہیں۔ آپ کو ثبوت فراہم کرنے ہوں گے۔

4. آپ کا ذاتی انٹرویو لیا جائے گا جس میں آپ سے تفصیل سے سوالات پوچھے جائیں گے۔ 

5. پھر آپ کی درخواست پر غور کیا جائے گا اور آپ کو فیصلے کی اطلاع دی جائے گی۔

6. آپ کسی بھی منفی فیصلے کے خلاف اپیل کر سکتے ہیں۔

امید ہے میں نے آپ کے سوال کا جواب دے دیا ہے۔ اگر آپ کو مزید رہنمائی کی ضرورت ہو تو مجھ سے پوچھیں۔ 

 ## Sources: 

 https://multecihaklari.info/ur/services/registration-and-status-6/?section=questions&question=5; https://multecihaklari.info/ur/services/unaccompanied-minors/?section=questions&question=21; https://multecihaklari.info/ur/services/registration-and-status-6/?section=questions&question=40; https://multecihaklari.info/ur/services/registration-and-status-6/?section=questions&question=32

In [7]:
def query_asylum_system(question):
    from IPython.display import Markdown, display
    # Invoke the LLM chain with the provided question
    response = rag_chain_with_source.invoke(question)

    # Extract the answer from the response
    answer = response["answer"]

    # Format the response using Markdown
    pretty_response = f"## Answer: \n\n{answer}\n\n## Sources:\n\n{extract_unique_urls(response)}"
    return pretty_response

In [8]:
Markdown(query_asylum_system(
    "کیا آپ مجھے ترکی میں ایک پناہ گزین کے طور پر میرے حقوق کے بارے میں بتا سکتے ہیں؟"))

## Answer: 

 جی ہاں، ترکی میں پناہ گزینوں کے کچھ اہم حقوق ہیں جن میں شامل ہیں:

- ترکی میں سیاسی پناہ کی درخواست دینے کا حق۔ اگر آپ کو اپنے ملک واپس جانے کا خوف ہے تو آپ عارضی تحفظ یا بین الاقوامی تحفظ کے لیے درخواست دے سکتے ہیں۔

- جب تک آپ کی درخواست کا جائزہ لیا جا رہا ہو، تب تک زبردستی واپس نہ بھیجے جانے کا حق۔ 

- عارضی تحفظ یا بین الاقوامی تحفظ کی حیثیت سے تعلیم، صحت کی دیکھ بھال اور سماجی امداد جیسی بنیادی خدمات تک رسائی حاصل کرنے کا حق۔

- کام کرنے اور کمائی کرنے کا حق، مشروط اس بات پر کہ آپ ورک پرمٹ حاصل کریں۔

یہ کچھ اہم حقوق ہیں جو ترکی میں پناہ گزینوں کو دیے جاتے ہیں۔ اگر آپ کے پاس مزید سوالات ہیں تو میں خوشی سے جواب دوں گا۔

## Sources:

https://multecihaklari.info/ur/services/unaccompanied-minors/?section=questions&question=12; https://multecihaklari.info/ur/services/registration-and-status-6/?section=questions&question=31; https://multecihaklari.info/ur/services/unaccompanied-minors/?section=questions&question=10; https://multecihaklari.info/ur/services/unaccompanied-minors/?section=questions&question=18